In [1]:
from calUtils import *
from scipy.optimize import least_squares, minimize

In [2]:
file_num = 18
folder = 'iProbeCalibration/geoCalRobot3'
def err_iprobe(iprobe_prm_input, solidCube_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                              iprobe_prm_input[1], 
                              iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)
    
def err_solidcube(solidCube_prm_input, iprobe_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                              iprobe_prm_input[1], 
                              iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)
    
def iterative_optimization(iprobe_prm_input, solidCube_prm_input, 
                           tol=1e-6, max_iter=1000):
    prev_error = np.inf
    for iteration in range(max_iter):
        result_iprobe = minimize(fun=err_iprobe, 
                                 x0=iprobe_prm_input, 
                                 args=(solidCube_prm_input, 'scalar'), 
                                 method='L-BFGS-B', 
                                 bounds=[(53.5, 73.5), 
                                         (67.0, 87.0), 
                                         (-87.0, -67.0)], 
                                 tol=tol,
                                 options={'maxiter': max_iter, 'disp': False})
        iprobe_prm_input = result_iprobe.x
        
        result_solidCube = least_squares(fun=err_solidcube, 
                                         x0=solidCube_prm_input, 
                                         args=(iprobe_prm_input, 'vector'), 
                                         ftol=tol, 
                                         # tr_options={'verbosity': 1, 'max_iter': 2000}, 
                                         verbose=0, 
                                         max_nfev=max_iter)
        solidCube_prm_input = result_solidCube.x
        
        current_error = err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')
        print(f"Iteration {iteration+1}, Error: {current_error:.8f}")
        if abs(prev_error - current_error) < tol:
            print("Converged.")
            break
        elif current_error >= prev_error:
            print("Converged.")
            return prev_iprobe_prm, prev_solidCube_prm, prev_error
        prev_error = current_error
        prev_iprobe_prm = iprobe_prm_input
        prev_solidCube_prm = solidCube_prm_input
    
    return iprobe_prm_input, solidCube_prm_input, current_error

In [ ]:
# test
iprobe_prm_input = [63.5, 77, -77]
solidCube_prm_input = [-2.2266e-03, 6.3928e-02, -5.1277e-05, 
                       5.4682e+00, 7.0481e-04, 4.8222e-04]
print(f"Before calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'vector')}")

iprobe_prm_input = [64.14325663027542, 75.53741941845027, -77.94238277637696]
solidCube_prm_input = [0.02938707937765631, 0.37415545856852334, -0.007216594687467497, 
                       5.499818818409724, -0.0069071979096573775, 0.0008926791868134888]
print(f"After calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'vector')}")


[-112.2184,	64.8964,	275.5654]
[-111.3495,	64.4109,	276.0666]
[-111.2519,	63.8615,	276.3714]
[-112.0633,	62.4341,	276.4679]
[-112.7899,	62.1124,	276.1603]
[-112.3527,	63.6411,	275.7657]
[-113.6366,	65.2966,	275.2977]
[-115.0387,	65.2724,	274.9361]
[-116.1574,	65.9929,	274.4903]
[-110.7948,	64.9088,	275.9589]
[-109.6972,	65.2718,	276.2281]
[-108.8437,	65.6034,	276.4031]
[-112.2078,	64.4554,	275.2865]
[-112.2803,	64.1573,	274.9495]
[-112.4212,	64.2950,	274.7378]
[-111.9702,	65.7611,	276.1355]
[-112.0673,	67.1079,	276.3809]
[-111.9304,	68.9565,	276.3161]
Average tip offset: [-112.1706,	64.9131,	275.7510]

[177.3106,	64.0310,	273.5989]
[178.1464,	62.3033,	273.7124]
[178.1858,	61.6144,	273.9060]
[177.3739,	61.2827,	274.4825]
[176.7194,	62.2903,	274.5372]
[177.1050,	63.8209,	273.7789]
[175.9018,	63.2826,	274.3535]
[174.3986,	63.3394,	275.0960]
[173.2721,	63.2132,	275.7232]
[178.7108,	64.6728,	273.0051]
[179.7752,	66.1419,	272.3185]
[180.5282,	67.5529,	271.6640]
[177.2667,	63.5696,	273.3238]


In [ ]:
iprobe_prm_input = [63.5, 77.0, -77.0]
solidCube_prm_input = [-2.2266e-03, 6.3928e-02, -5.1277e-05, 
                       5.4682e+00, 7.0481e-04, 4.8222e-04]

print(f"Before calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')}")
final_iprobe_prm, final_solidCube_prm, final_error = iterative_optimization(iprobe_prm_input, solidCube_prm_input)

print("\nFinal optimized parameters:")
print(f"iProbe Parameters: [{final_iprobe_prm[0]}, {final_iprobe_prm[1]}, {final_iprobe_prm[2]}]")
print(f"SolidCube Parameters: [{final_solidCube_prm[0]}, {final_solidCube_prm[1]}, {final_solidCube_prm[2]}, {final_solidCube_prm[3]}, {final_solidCube_prm[4]}, {final_solidCube_prm[5]}]")
print("Final Error:", final_error)